In [ ]:
%pip install -qU  langchain_milvus
%pip install -qU langchain-openai

In [6]:
import os
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_milvus import Milvus


# The easiest way is to use Milvus Lite where everything is stored in a local file.
# If you have a Milvus server you can use the server URI such as "http://localhost:19530".
URI = "http://localhost:19530"

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=os.environ["OPENAI_API_KEY"])

docs = [
    Document(metadata={'source': 'example.pdf', 'page': 0, 'author': 'Jorge Angel Juarez Vazquez', 'namespace': 'jorgeang33@gmail.com'}, page_content='Titulo\nEncabezado 1\nPara empezar ahora mismo, pulse el texto de cualquier marcador de posición (como este, por\nejemplo) y comience a escribir.\nPara aplicar facilmente cualquier formato de texto que vea en esta pagina, vaya al grupo\nEstilos, que encontrara en la pestana Inicio de la cinta de opciones.\n¢ Desea insertar una imagen de sus archivos o agregar una forma o un cuadro de texto?\n¡Adelante! En la pestaña Insertar de la cinta de opciones, pulse la opción que necesite.\n'),
    Document(metadata={'source': 'example.pdf', 'page': 1, 'author': 'Jorge Angel Juarez Vazquez', 'namespace': 'jorgeang33@gmail.com'}, page_content='Esto es texto que esta en una imagen, en la segunda\npagina del documento, a continuación, información sobre\nlos perros:\nEl perro, llamado perro doméstico o can, y en algunos\nlugares coloquialmente llamado chucho, tuso, choco,\nentre otros; es un mamifero carnivoro de la familia de los\ncanidos, que constituye una especie del género Canis.\n')
]
vectorstore = Milvus.from_documents(
    docs, 
    embeddings,
    collection_name="other_name",
    connection_args={"uri": URI},
    drop_old=True,
    auto_id=False,
    primary_field="pk", 
    partition_key_field="namespace",  # Use the "namespace" field as the partition key
    index_params={"metric_type": "COSINE"}
)

In [25]:
for doc in docs:
    print(doc.metadata)

{'source': 'example.pdf', 'page': 0, 'author': 'Jorge Angel Juarez Vazquez', 'namespace': 'jorgeang33@gmail.com'}
{'source': 'example.pdf', 'page': 1, 'author': 'Jorge Angel Juarez Vazquez', 'namespace': 'jorgeang33@gmail.com'}


In [4]:
import json

uuids = [str(hash(json.dumps(doc.metadata))) for doc in docs]
print(uuids)

vectorstore.add_documents(docs, ids=uuids)

['-2014741237850642074', '8981368537327681524']


['-2014741237850642074', '8981368537327681524']

In [ ]:
import os
import json

from typing import Optional, List, Any

from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_milvus import Milvus


class MilvusStorage:
    def __init__(self, collection_name):
        self.milvus_url = os.getenv("MILVUS_URL")
        self.embeddings_model = OpenAIEmbeddings(model=os.getenv("EMBEDDINGS_MODEL"), api_key=os.getenv("OPENAI_API_KEY"))
        self.collection_name = collection_name
    
    def create_collection(self, partition_key_field: str = "namespace") -> Optional[Milvus]:
        try:
            vectorstore = Milvus(
                self.embeddings_model,
                collection_name=self.collection_name,
                connection_args={"uri": self.milvus_url},
                drop_old=True,
                auto_id=False,
                primary_field="pk", 
                index_params={"metric_type": "COSINE"}, 
                partition_key_field=partition_key_field # Use the "namespace" field as the partition key
            )
            return vectorstore
        except Exception as e:
            print(f"Error creating collection: {e}")
            return None

    def insert_files(self, vectorstore: Milvus, docs: List[Document]) -> Optional[List[str]]:
        try:
            uuids = [str(hash(json.dumps(doc.metadata))) for doc in docs]
            ids = vectorstore.add_documents(docs, ids=uuids)
            return ids
        except Exception as e:
            print(f"Error inserting documents: {e}")
        

    def create_retriver(self, vectorstore: Milvus, namespace: str) -> Any:
        try:
            retriver = vectorstore.as_retriever(
                search_kwargs = {
                    "expr": f"namespace: == '{namespace}'", 
                    "k": 2, 
                    'metric_type': "COSINE",
                    'score_threshold': 0.8
                }
            )
            return retriver
        except Exception as e:
            print(f"Error creating RAG: {e}")